<a href="https://colab.research.google.com/github/T-Sunm/Email-Classification/blob/main/Email_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install gdown


In [ ]:
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import gdown

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!gdown --id 1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R
To: /kaggle/working/2cls_spam_text_cls.csv
100%|████████████████████████████████████████| 486k/486k [00:00<00:00, 94.8MB/s]


In [ ]:
DATASET_PATH = "/kaggle/working/2cls_spam_text_cls.csv"
df = pd.read_csv(DATASET_PATH)
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
messages = df['Message'].values.tolist()
labels = df['Category'].values.tolist()

print("messages", messages[:5])
print("\n")
print("labels", labels[:5])

messages ['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'U dun say so early hor... U c already then say...', "Nah I don't think he goes to usf, he lives around here though"]


labels ['ham', 'ham', 'spam', 'ham', 'ham']


# Data Preprocessing : Bag of Words

![image.png](attachment:bbac5891-29a5-42f1-9794-e9976c4b08f7.png)

## Lower case

In [ ]:
def lowercase(text:str):
    return text.lower()
INPUT_TEXT = "The cat's toys are scattered everywhere!"
lowercase(INPUT_TEXT)

"the cat's toys are scattered everywhere!"

## Punctuation Removal

translator = str.maketrans('', '', string.punctuation)
- str.maketrans là một phương thức của lớp str dùng để tạo bảng dịch.
- Tham số đầu tiên và thứ hai (cả hai là chuỗi rỗng) chỉ ra rằng không có ký tự nào được thay thế bằng ký tự khác.
- Tham số thứ ba, string.punctuation, chứa tất cả các dấu câu. Các ký tự trong chuỗi này sẽ bị loại bỏ khi áp dụng bảng dịch này.


In [ ]:
def punctuation_removal(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

INPUT_TEXT = "The cat's toys are scattered everywhere!"
INPUT_TEXT = lowercase(INPUT_TEXT)
punctuation_removal(INPUT_TEXT)

'the cats toys are scattered everywhere'

## Tokenize

In [ ]:
def tokenize(text):
    return text.split()

INPUT_TEXT = "The cat's toys are scattered everywhere!"
INPUT_TEXT = lowercase(INPUT_TEXT)
INPUT_TEXT = punctuation_removal(INPUT_TEXT)
tokenize(INPUT_TEXT)

['the', 'cats', 'toys', 'are', 'scattered', 'everywhere']

## Remove Stopword

trong tiếng Anh, các từ như "the", "is", "in", "at", "of" là các stop words. Mục tiêu của việc loại bỏ các từ này là để giảm kích thước của văn bản và tập trung vào các từ mang ý nghĩa quan trọng hơn.

In [ ]:
def remove_stopwords(tokens):
    stop_words = nltk.corpus.stopwords.words('english')
    return [token for token in tokens if token not in stop_words]

INPUT_TEXT = "The cat's toys are scattered everywhere!"
INPUT_TEXT = lowercase(INPUT_TEXT)
INPUT_TEXT = punctuation_removal(INPUT_TEXT)
tokens = tokenize(INPUT_TEXT)
remove_stopwords(tokens)

['cats', 'toys', 'scattered', 'everywhere']

## Stemming

Stemming: Rút gọn các từ còn lại về dạng gốc của chúng. Trong ví dụ này, "cats" trở thành "cat", "scattered" trở thành "scatter", kết quả là ["cat", "toy", "scatter","everywhere"].

In [ ]:
def stemming(tokens):
    stemmer = nltk.PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

INPUT_TEXT = "The cat's toys are scattered everywhere!"
INPUT_TEXT = lowercase(INPUT_TEXT)
INPUT_TEXT = punctuation_removal(INPUT_TEXT)
tokens = tokenize(INPUT_TEXT)
tokens = remove_stopwords(tokens)
stemming(tokens)

['cat', 'toy', 'scatter', 'everywher']

## Final

In [ ]:
def preprocess_text(text):
    text = lowercase (text)
    text = punctuation_removal(text)
    tokens = tokenize (text)
    tokens = remove_stopwords (tokens)
    tokens = stemming (tokens)

    return tokens

INPUT_TEXT = "The cat's toys are scattered everywhere!"
preprocess_text(INPUT_TEXT)

['cat', 'toy', 'scatter', 'everywher']

In [ ]:
messages_prep = [preprocess_text(mess) for mess in messages]
print(messages_prep[:5])

[['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat'], ['ok', 'lar', 'joke', 'wif', 'u', 'oni'], ['free', 'entri', '2', 'wkli', 'comp', 'win', 'fa', 'cup', 'final', 'tkt', '21st', 'may', '2005', 'text', 'fa', '87121', 'receiv', 'entri', 'questionstd', 'txt', 'ratetc', 'appli', '08452810075over18'], ['u', 'dun', 'say', 'earli', 'hor', 'u', 'c', 'alreadi', 'say'], ['nah', 'dont', 'think', 'goe', 'usf', 'live', 'around', 'though']]


# Build Vocabulary

In [ ]:
def create_dictionary(messages):
    dictionary = []
    for mess in messages:
        for vocab in mess:
            if vocab not in dictionary:
                dictionary.append(vocab)

    return dictionary

dictionary = create_dictionary(messages_prep)
print(np.array(dictionary).shape)

(8190,)


# Create feature

In [ ]:
def create_features(mess, dictionary):
    tokenizer_mess = np.zeros(len(dictionary))
    for vocab_mess in mess:
        if vocab_mess in dictionary:
            position = dictionary.index(vocab_mess)
            tokenizer_mess[position] += 1

    return tokenizer_mess

X = np.array([create_features(mess, dictionary) for mess in messages_prep])
print(X)

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Label Encoder

In [ ]:
le = LabelEncoder()
y = le.fit_transform(labels)

print("Classes :" ,le.classes_)
print("Encoder Labels :", y)

Classes : ['ham' 'spam']
Encoder Labels : [0 0 1 ... 0 0 0]


# Split train, test

### Chia dữ liệu theo tỷ lệ 7/2/1 (train /val /test)
Dữ liệu ban đầu có 1000 mẫu
- Tập huấn luyện và xác thực:
    - Số lượng mẫu của tập xác thực (X_val, y_val) = 1000 * 0.2 = 200 mẫu.
    - Số lượng mẫu của tập huấn luyện ban đầu = 1000 * 0.8 = 800 mẫu.
- Tập huấn luyện cuối cùng và tập kiểm tra:
    - Số lượng mẫu của tập kiểm tra (X_test, y_test) = 800 * 0.125 = 100 mẫu.
    - Số lượng mẫu của tập huấn luyện cuối cùng (X_train, y_train) = 800 * 0.875 = 700 mẫu.

In [ ]:
VAL_SIZE = 0.2
VAL_TEST = 0.125
SEED = 0
IS_SHUFFLE = True

X_train, X_val, y_train, y_val = train_test_split(
    X,y,
    test_size = VAL_SIZE,
    shuffle = IS_SHUFFLE,
    random_state = SEED
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train,y_train,
    test_size = VAL_TEST,
    shuffle = IS_SHUFFLE,
    random_state = SEED
)

print(f'Number of training samples: {X_train.shape[0]}')
print(f'Number of val samples: {X_val.shape[0]}')
print(f'Number of test samples: {X_test.shape[0]}')

Number of training samples: 3899
Number of val samples: 1115
Number of test samples: 558


# Train model

In [ ]:
%%time

model = GaussianNB()
print("start training ...")
model = model.fit(X_train, y_train)
print("Training completed")

start training ...
Training completed
CPU times: user 309 ms, sys: 125 ms, total: 435 ms
Wall time: 436 ms


# Evaluation

In [ ]:
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)
val_accurancy = accuracy_score(y_val, y_val_pred)
test_accurancy = accuracy_score(y_test, y_test_pred)

print("Độ chính xác validation gốc - validation predict", val_accurancy)
print("Độ chính xác test gốc - test predict", val_accurancy)

Độ chính xác validation gốc - validation predict 0.8816143497757848
Độ chính xác test gốc - test predict 0.8816143497757848


# Prediction

In [ ]:
def predict(text, model, dictionary):
    prep_text = preprocess_text(text)

#     reshape vì quy chuẩn của sklearn bắt buộc vậy
    x = create_features(prep_text, dictionary).reshape(1, -1)
    y = model.predict(x)
    print(y)

    y_cls = le.inverse_transform(y)
    return y_cls

test_input = 'I am actually thinking a way of doing something useful'
prediction_cls = predict(test_input, model, dictionary)
print(f'Prediction: {prediction_cls}')

[0]
Prediction: ['ham']
